<a href="https://colab.research.google.com/github/dajopr/lectures/blob/main/image_processing/lecture_03_keypoint_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenCV Exercise Sheet: Feature extraction - Corners and blobs

Prerequisites: Basic knowledge of Python, NumPy, OpenCV (reading/displaying/converting images),
               Spatial Filtering concepts, Edge Detection concepts, Matplotlib.
Goal: This exercise sheet focuses on detecting key features like corners and blobs
      in images using OpenCV, understanding detector parameters, and comparing methods.

In [ ]:
# %% Import necessary libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os # Used for checking file paths

# %% Helper function to display images (Adapted from lecture_02_spatial_filtering.py)
def show_images(images, titles, rows, cols, figsize=(15, 10), main_title="Image Comparison"):
    """Helper function to display multiple images using Matplotlib"""
    if len(images) != len(titles):
        print("Warning: Number of images and titles do not match.")
        # Attempt to proceed with the minimum of the two counts
        min_count = min(len(images), len(titles))
        images = images[:min_count]
        titles = titles[:min_count]

    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    fig.suptitle(main_title, fontsize=16)

    # Handle case where subplot returns a single Axes object or 1D array
    if rows * cols == 1:
        axes = np.array([axes])
    axes = axes.ravel() # Flatten the axes array

    plot_index = 0
    for i, img in enumerate(images):
        if plot_index < len(axes): # Ensure we don't exceed the number of axes
            current_ax = axes[plot_index]
            if img is not None:
                # Check if image is color or grayscale
                if len(img.shape) == 2 or (len(img.shape) == 3 and img.shape[2] == 1):
                    # Display grayscale image
                    current_ax.imshow(img, cmap='gray')
                elif len(img.shape) == 3 and img.shape[2] == 3:
                    # Assume BGR format from OpenCV, convert to RGB for Matplotlib
                    current_ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                elif len(img.shape) == 3 and img.shape[2] == 4:
                     # Assume BGRA format from OpenCV, convert to RGBA for Matplotlib
                    current_ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGRA2RGBA))
                else:
                    current_ax.set_title(f'{titles[i]} (Unsupported Format)')
                    print(f"Warning: Image '{titles[i]}' has an unsupported shape: {img.shape}")

                if len(img.shape) != 3 or img.shape[2] != 4: # Don't overwrite title if unsupported format message was set
                     current_ax.set_title(titles[i])

            else:
                current_ax.set_title(f'{titles[i]} (None)') # Handle None images
            current_ax.axis('off') # Hide axes ticks
            plot_index += 1
        else:
            print(f"Warning: More images provided than subplot slots ({rows*cols}). Skipping '{titles[i]}'.")


    # Hide any unused subplots
    for j in range(plot_index, len(axes)):
        axes[j].axis('off')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to prevent title overlap
    plt.show()

##  Exercise 1: Basic Harris Corner Detection

Objective: Understand and apply the Harris corner detector using OpenCV.
            Visualize the detected corners on an image.

Instructions:
1. Load the chessboard image (`IMG_PATH_CHESS`) or shapes image (`IMG_PATH_SHAPES`).
    Handle potential loading errors.
2. Convert the loaded image to grayscale.
3. Apply the `cv2.cornerHarris()` function to the grayscale image.
    - Use parameters: `blockSize=2`, `ksize=3`, `k=0.04`.
    - Remember `cv2.cornerHarris` requires a float32 input image.
4. The result `dst` from `cornerHarris` is a float image with corner response scores.
   Threshold this result to identify strong corners. A common approach is:
   `threshold = 0.01 * dst.max()`
   Corners are pixels where `dst > threshold`.
5. Create a copy of the original *color* image to draw on.
6. Mark the detected corners on the color image copy. You can do this by:
   a) Setting the pixel color directly: `img_draw[dst > threshold] = [255, 0, 0]` (Marks in Blue for BGR)
   b) Finding the coordinates `(x, y)` where `dst > threshold` (e.g., using `np.where`)
   and then drawing circles at these coordinates using `cv2.circle()`. Method (b) is often clearer.
7. Display the original image and the image with detected corners using the `show_images` function.

### Discussion Questions (Exercise 1):
1. What happens visually if you increase the `k` parameter in `cv2.cornerHarris()` (e.g., to 0.1)? What if you decrease it (e.g., to 0.01)?
2. What is the effect of changing the `blockSize` parameter? Try `blockSize=5`.
3. Looking at the `chessboard.png`, does Harris find corners mostly where you expect them? Are there any surprises?

## Exercise 2: Shi-Tomasi ("Good Features to Track") vs. Harris
Objective: Compare the Harris corner detector with the Shi-Tomasi detector (`cv2.goodFeaturesToTrack`).
           Understand the parameters and output differences.
Instructions:
1. Load the same image used in Exercise 1 (`IMG_PATH_CHESS` or `IMG_PATH_SHAPES`).
2. Convert it to grayscale.
3. Detect corners using `cv2.cornerHarris()` as you did in Exercise 1. Draw these corners
   on a copy of the original color image (e.g., using blue circles).
4. Detect corners using `cv2.goodFeaturesToTrack()` on the grayscale image.
   - Use parameters like `maxCorners=100`, `qualityLevel=0.01`, `minDistance=10`.
   - This function directly returns the corner coordinates. Remember to convert them to integers
     (e.g., using `np.intp()` or `np.int0()`) before drawing.
5. Draw the Shi-Tomasi corners on a *different* copy of the original color image (e.g., using green circles).
6. Display the Harris results and Shi-Tomasi results side-by-side using `show_images`.

### Discussion Questions (Exercise 2):
1. Compare the *distribution* of corners found by Harris (after thresholding) and Shi-Tomasi. How does `minDistance` in Shi-Tomasi affect this?
2. How does changing the `qualityLevel` in Shi-Tomasi affect the *number* of corners found?
3. Based on the visual results, which detector seems to give more evenly spaced, well-defined corners, potentially better suited for tracking applications? Why?

## Exercise 3: Basic Blob Detection with `SimpleBlobDetector`
Objective: Learn to use OpenCV's `SimpleBlobDetector` with its default settings
           to find blobs (regions of similar intensity) in an image.
Instructions:
1. Load the `dots.png` image (`IMG_PATH_DOTS`). It's often best to load it directly as grayscale
   using `cv2.imread(path, cv2.IMREAD_GRAYSCALE)`. Handle loading errors.
2. Create an instance of the `cv2.SimpleBlobDetector` using the default parameters:
   `params = cv2.SimpleBlobDetector_Params()`
   `detector = cv2.SimpleBlobDetector_create(params)`
   *(Note: For older OpenCV versions < 3, it might be `detector = cv2.SimpleBlobDetector(params)`)*
3. Detect blobs in the grayscale image using `keypoints = detector.detect(image)`.
4. The result `keypoints` is a list of `cv2.KeyPoint` objects.
5. Use `cv2.drawKeypoints()` to draw the detected blobs onto the image.
   - Use the flag `cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS` to draw circles
     representing the size and location of the blobs.
   - Example: `img_with_keypoints = cv2.drawKeypoints(image, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)` (Draws
6. Display the original grayscale image and the image with detected blobs using `show_images`.

### Discussion Questions (Exercise 3):
1. Did the default detector find most/all of the dots? Did it find anything unexpected?
2. What information does a `cv2.KeyPoint` object contain? (Hint: try `print(keypoints[0].pt, keypoints[0].size)` if `keypoints` is not empty).
3. The default detector looks for dark blobs on a light background if you feed it a standard grayscale image. How could you detect light blobs on a dark background?

## Exercise 4: Filtering Blobs by Properties
Objective: Configure `SimpleBlobDetector` parameters to filter detected blobs based on
           properties like Area and Circularity.
Instructions:
1. Load the `various_blobs.png` image (`IMG_PATH_BLOBS`) as grayscale. This image should
   contain blobs of different sizes and shapes. Handle loading errors.
2. Create a `cv2.SimpleBlobDetector_Params()` object.
3. Modify the parameters object to filter blobs:
   - **Filter by Area:**
     - `params.filterByArea = True`
     - `params.minArea = 100` (Example: Adjust based on your image)
     - `params.maxArea = 2000` (Example: Adjust based on your image)
   - **Filter by Circularity:** (Circularity = 1 for a perfect circle)
     - `params.filterByCircularity = True`
     - `params.minCircularity = 0.7` (Example: Find blobs that are mostly round)
     - `params.maxCircularity = 1.0` (Usually okay unless excluding circles)
   - *Experiment with these values to see their effect.*
4. Create the detector using these modified parameters: `detector = cv2.SimpleBlobDetector_create(params)`.
5. Detect blobs in the image: `keypoints = detector.detect(image)`.
6. Draw the detected (filtered) blobs onto the image using `cv2.drawKeypoints` (like in Ex 3).
7. Display the original grayscale image and the image with filtered blobs using `show_images`.

### Discussion Questions (Exercise 4):
1. How did setting `minArea` and `maxArea` change which blobs were detected compared to Exercise 3 (or default parameters)?
2. What was the effect of adding the `minCircularity` filter? Which shapes were excluded?
3. Why is it often necessary to tune multiple filter parameters together (e.g., Area *and* Circularity) to isolate specific types of blobs effectively?

## Bonus Exercise: Sub-Pixel Corner Refinement
Objective: Improve the accuracy of corner locations detected by Shi-Tomasi
           using `cv2.cornerSubPix` to achieve sub-pixel precision.
Instructions:
1. Load the `chessboard.png` image (`IMG_PATH_CHESS`) and convert to grayscale.
2. Detect initial corner estimates using `cv2.goodFeaturesToTrack`. Store these corners
   (they should be float32, which is the default output).
3. Define the termination criteria for the sub-pixel refinement process. This tells the
   algorithm when to stop iterating. A common criteria is:
   `criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)`
   (Stop after 30 iterations or when accuracy `EPS`=0.001 is reached).
4. Use `cv2.cornerSubPix()` to refine the initial corner locations.
   - Inputs needed: grayscale image, initial corners (float32), `winSize` (search window size, e.g., `(11, 11)`),
     `zeroZone` (usually `(-1, -1)` to ignore), and the `criteria`.
   - `corners_refined = cv2.cornerSubPix(gray, corners_initial, winSize=(11, 11), zeroZone=(-1, -1), criteria=criteria)`
5. Create a copy of the original color image to draw on.
6. Draw both the initial corners (converted to *integers* for drawing, e.g., green circles)
   and the refined corners (using their *float* coordinates rounded/cast to int for drawing, e.g., smaller red circles or crosses).
7. Display the result. Use `plt.xlim()` and `plt.ylim()` on the Matplotlib plot to zoom
   in on a specific corner region to visually inspect the difference between initial
   and refined locations.